In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

FILE_TO_PLOT = "./ufal_emmt/preprocessed-data/eeg/See/P43-32-S191-A-U.csv"

# Load CSV file
def plot_eeg_timeseries(csv_file):
    # Read the CSV
    df = pd.read_csv(csv_file)
    
    # Extract time series
    time = df['TimeStamp']
    
    # Get electrode columns (Delta to RAW)
    eeg_columns = df.columns[1:21]  # Delta_TP9 to RAW_TP10
    
    # Plot each electrode signal
    plt.figure(figsize=(25, 8))
    for col in eeg_columns:
        plt.plot(time, df[col], label=col)
    
    plt.xlabel("Time (s)")
    plt.ylabel("EEG Signal Value")
    plt.title("EEG Time Series for All Electrodes")
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid()
    plt.show()

# Example usage
csv_file = FILE_TO_PLOT  # Change to the actual file path
plot_eeg_timeseries(csv_file)


# MNE

In [ ]:
import pandas as pd
import numpy as np
import mne

def load_eeg_to_mne(csv_file):
    # Read the CSV file
    df = pd.read_csv(csv_file)
    
    # Convert TimeStamp from HH:MM:SS.sss format to seconds
    df['TimeStamp'] = pd.to_datetime(df['TimeStamp'], format='%H:%M:%S.%f', errors='coerce')
    df['TimeStamp'] = (df['TimeStamp'] - df['TimeStamp'].iloc[0]).dt.total_seconds()
    
    # Forward-fill missing data
    df.fillna(method='ffill', inplace=True)
    
    # Ensure no NaN values remain by filling any remaining ones with 0
    df.fillna(0, inplace=True)
    time = df['TimeStamp'].values
    
    # Select EEG columns (Delta to RAW)
    eeg_columns = df.columns[1:21]  # Delta_TP9 to RAW_TP10
    eeg_data = df[eeg_columns].values.T  # Transpose to match (channels, samples)
    
    # Define sampling frequency (estimate based on time diffs)
    sfreq = 1 / np.mean(np.diff(time)) if len(time) > 1 else 256  # Default to 256 if only one timestamp
    
    # Create MNE info structure
    ch_names = list(eeg_columns)
    ch_types = ['eeg'] * len(ch_names)
    info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
    
    # Create RawArray object
    raw = mne.io.RawArray(eeg_data, info)
    return raw

# Example usage
csv_file = FILE_TO_PLOT  # Change to the actual file path
raw = load_eeg_to_mne(csv_file)
raw.plot(scalings="auto");  # Visualize the EEG data
